In [18]:
%run 02-connect.ipynb

In [19]:
%%sql

USE classicmodels

""


In [14]:
%%sql

WITH customers_in_usa AS (
    SELECT customerName,
        state
    FROM customers
    WHERE country = 'USA'
)
SELECT customerName
FROM customers_in_usa
WHERE state = 'CA'
ORDER BY customerName;

,customerName
0,Boards & Toys Co.
1,Collectable Mini Designs Co.
2,Corporate Gift Ideas Co.
3,"Men 'R' US Retailers, Ltd."
4,Mini Gifts Distributors Ltd.
5,Mini Wheels Co.
6,Signal Collectibles Ltd.
7,Technics Stores Inc.
8,The Sharp Gifts Warehouse
9,Toys4GrownUps.com


In this example, the name of the CTE is `customers_in_usa`, the query that defines the CTE returns two columns `customerName` and `state`. Hence, the `customers_in_usa` CTE returns all customers located in the USA.

After defining the `customers_in_usa` CTE, we referenced it in the `SELECT` statement to select only customers located in California.

In [15]:
%%sql

WITH topsales2003 AS (
    SELECT salesRepEmployeeNumber employeeNumber,
        SUM(quantityOrdered * priceEach) sales
    FROM orders
        INNER JOIN orderdetails USING (orderNumber)
        INNER JOIN customers USING (customerNumber)
    WHERE YEAR(shippedDate) = 2003
        AND status = 'Shipped'
    GROUP BY salesRepEmployeeNumber
    ORDER BY sales DESC
    LIMIT 5
)
SELECT employeeNumber,
    firstName,
    lastName,
    sales
FROM employees
    JOIN topsales2003 USING (employeeNumber);

,employeeNumber,firstName,lastName,sales
0,1165,Leslie,Jennings,413219.85
1,1370,Gerard,Hernandez,295246.44
2,1401,Pamela,Castillo,289982.88
3,1621,Mami,Nishi,267249.40
4,1501,Larry,Bott,261536.95


In this example, the CTE returns the top 5 sales rep in 2003. After that, we referenced to the `topsales2003` CTE to get additional information about the sales rep including first name and last name.

In [20]:
%%sql

WITH salesrep AS (
    SELECT employeeNumber,
        CONCAT(firstName, ' ', lastName) AS salesrepName
    FROM employees
    WHERE jobTitle = 'Sales Rep'
),
customer_salesrep AS (
    SELECT customerName,
        salesrepName
    FROM customers
        INNER JOIN salesrep ON employeeNumber = salesrepEmployeeNumber
)
SELECT *
FROM customer_salesrep
ORDER BY customerName;

,customerName,salesrepName
0,Alpha Cognac,Gerard Hernandez
1,American Souvenirs Inc,Foon Yue Tseng
2,Amica Models & Co.,Pamela Castillo
3,"Anna's Decorations, Ltd",Andy Fixter
4,Atelier graphique,Gerard Hernandez
...,...,...
95,"UK Collectables, Ltd.",Larry Bott
96,"Vida Sport, Ltd",Martin Gerard
97,Vitachrome Inc.,Foon Yue Tseng
98,"Volvo Model Replicas, Co",Barry Jones


In this example, we have two CTEs in the same query. The first CTE ( `salesrep`) gets the employees whose job titles are the sales representative. The second CTE ( `customer_salesrep` ) references the first CTE in the [`INNER JOIN`](https://www.mysqltutorial.org/mysql-inner-join.aspx) clause to get the sales rep and customers of whom each sales rep is in charge.

After having the second CTE, we query data from that CTE using a simple `SELECT` statement with the [`ORDER BY`](https://www.mysqltutorial.org/mysql-order-by/) clause.